In [ ]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='1mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))

# Add some additional features to the plot

fplt.show()

In [ ]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot

fplt.show()

In [ ]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot
mean_volume = chamada_api['Volume'].mean()
fplt.add_line(axv,mean_volume, color='#4f4f4f', style='--')
fplt.show()

In [2]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='5mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot
fplt.plot(df_renko.Volume.ewm(span=24).mean(), ax=axv, color=1)
fplt.show()